In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import warnings

# Check event stats

In [ ]:
fn = os.path.join('/scratch/cascade/UEH-daily/stats/wbgtmax30_STATS.json')

In [ ]:
df = pd.read_json(fn, orient = 'split')

In [ ]:
df.head()

In [ ]:
# Dehli 6955, Kolkata 9691

In [ ]:
def mk_data(df, id_):
    data = df[df['ID_HDC_G0'] == id_][['year','duration']].groupby('year').sum()
    pd.DataFrame(data.reset_index(inplace = True))
    
    return data

In [ ]:
dehli = mk_data(df, 6955)
kolkata = mk_data(df, 9691)

In [ ]:
fig = plt.gcf()
fig.set_size_inches(10, 10)
plt.plot(dehli['year'], dehli['duration'], color = 'cornflowerblue', linewidth = 3)
plt.plot(kolkata['year'], kolkata['duration'], color = 'chocolate', linewidth = 3)
plt.legend(['Deli','Kolata'], fontsize = 15)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('days/yr wbgt-max > 30°C', fontsize = 15)

In [ ]:
#### Args
DATA = 'wbgt30' # Always update WBGT32_1D, WBGT28_1D, HI406_1D HI406_2D & HI461_1D 
DATA_PATH = os.path.join('/scratch/cascade/UEH-daily/stats/')
FN_POP = os.path.join('/home/cascade/projects/UrbanHeat/data/interim/GHS-UCDB-Interp.csv')
FN_STATS = os.path.join(DATA_PATH, DATA+'_STATS.json')
FN_OUT = os.path.join(DATA_PATH, DATA+'_EXP.json') 

In [ ]:
FN_STATS

In [ ]:
FN_STATS = os.path.join('/scratch/cascade/UEH-daily/stats/wbgtmax30_STATS.json')

In [ ]:
stats = pd.read_json(FN_STATS, orient = 'split') # read in stats
df_pop = pd.read_csv(FN_POP) # read in interp population from GHS-UCDB

In [ ]:
len(np.unique(stats['ID_HDC_G0']))

In [ ]:
stats

In [ ]:
def tot_days(df):
    """ Calulates the total number of days per year when a heat threshold was met
    """
    df_out = df[['ID_HDC_G0','year','duration']].groupby(['ID_HDC_G0','year']).sum().reset_index()
    df_out.rename(columns={'duration':'tot_days'}, inplace=True)
    
    return df_out

In [ ]:
#### Args
DATA = 'wbgtmax30' # Always update WBGT32_1D, WBGT28_1D, HI406_1D HI406_2D & HI461_1D 
DATA_PATH = os.path.join('/scratch/cascade/UEH-daily/stats/')
FN_POP = os.path.join('/home/cascade/projects/UrbanHeat/data/interim/GHS-UCDB-Interp.csv')
FN_STATS = os.path.join(DATA_PATH, DATA+'_STATS.json')
FN_OUT = os.path.join(DATA_PATH, DATA+'_EXP.json') 

In [ ]:
df = pd.read_json(FN_OUT, orient = 'split')

In [ ]:
df

# Check TREND

In [7]:
fn = '/scratch/cascade/UEH-daily/stats/wbgtmax30_TREND_ALL.json'

In [8]:
trend = pd.read_json(fn, orient = 'split')
len(trend)

6332

In [9]:
trend.head()

,ID_HDC_G0,coef_pdays,p_value_pdays,coef_heat,p_value_heat,coef_pop,p_value_pop,coef_totDays,p_value_totDays,coef_attrib,coef_attrib_norm,CTR_MN_NM,UC_NM_MN,GCPNT_LAT,GCPNT_LON,region,sub-region,intermediate-region,P1983,P2016
22,52,101604.735492,0.0007,28022.279616,0.0764,73582.455875,0.0,0.280825,0.0764,0.448406,0.748042,Mexico,La Paz [MEX],24.117658,-110.310119,Americas,Latin America and the Caribbean,Central America,99785.538677,249632.317993
26,57,531741.922626,0.0002,-36648.428326,0.6062,568390.350952,0.0,-0.084798,0.6062,1.137843,0.748998,Mexico,Mexicali [MEX]; Calexico [USA],32.625439,-115.447658,Americas,Latin America and the Caribbean,Central America,432187.321425,805239.718513
28,59,178763.571718,0.0000,3086.565631,0.9498,175677.006087,0.0,0.036364,0.9498,0.965468,0.748759,Mexico,San Luis Rio Colorado [MEX]; San Luis [USA],32.456882,-114.768297,Americas,Latin America and the Caribbean,Central America,84880.554856,195368.303088
29,60,52328.458359,0.0000,1349.022243,0.9496,50979.436116,0.0,0.032544,0.9496,0.948440,0.748735,United States,Yuma [USA],32.693505,-114.638648,Americas,Northern America,Northern America,41452.350136,78222.818237
30,61,124447.852339,0.0000,6867.884309,0.0169,117579.968031,0.0,0.487701,0.0169,0.889626,0.748654,Mexico,Puerto Penasco [MEX],31.325557,-113.529039,Americas,Latin America and the Caribbean,Central America,14082.175062,66491.710266


In [ ]:
ids_trend = np.unique(trend['ID_HDC_G0'])

In [ ]:
fn = '/scratch/cascade/UEH-daily/stats/wbgtmax30_EXP.json'

In [ ]:
exp = pd.read_json(fn, orient = 'split')

In [ ]:
ids_exp = np.unique(exp['ID_HDC_G0'])

In [ ]:
len(ids_exp)

In [ ]:
len(ids_trend)

In [ ]:
#missing = list(set(ids_exp) - set(ids_trend))

In [ ]:
test = exp[exp['ID_HDC_G0'] == 8203]


In [ ]:
pop = OLS(df = test, geog = 'ID_HDC_G0', col = 'people_days_pop', alpha = 0.05) # get stats 


In [ ]:
pop

In [ ]:
for item in list(zip(trend['ID_HDC_G0'],trend['coef_pop'])):
    print(item)

In [ ]:
exp[exp['ID_HDC_G0'] == 733]

# WBGT

In [ ]:
# Functions
def c_to_f(C):
    "Convert HI C to F"
    return 1.8 * C + 32

def F_to_C(Tmax_F):
    "Function converts temp in F to C"
    Tmax_C = (Tmax_F - 32) * (5/9)
    
    return Tmax_C

def hi_to_wbgt(HI):
    """ Convert HI to WBGT using emprical relationship from Bernard and Iheanacho 2015
    WBGT [◦C] = −0.0034 HI2 + 0.96 HI−34; for HI [◦F]
    Args:
        HI = heat index as an array
    """
    
    WBGT = -0.0034*HI**2 + 0.96*HI - 34
    
    return WBGT

In [ ]:
hi_to_wbgt(105)

In [ ]:
c_to_f(29)